<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/Ka-Boom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Extended impact + crater model (allowing up to ~2000 km craters)
# Same gravity-dominated scaling, but with larger slider ranges
# K scaling:
# For hard, competent rock: K ≈ 2.0–2.2
# For generic rocky regolith: K ≈ 1.6–1.9
# For sand or unconsolidated soils: K ≈ 1.0–1.3
# For ice targets: K ≈ 0.8–1.2 (lower if porous, comet-like)
# For layered sediments: K ≈ 1.0–1.5

import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

# Conversions
G_PER_CM3_TO_KG_PER_M3 = 1000.0
KM_TO_M  = 1000.0
KMS_TO_MS = 1000.0
J_PER_KT_TNT = 4.184e12
J_PER_MT_TNT = 4.184e15
HIROSHIMA_KT = 15.0
J_PER_HIRO = HIROSHIMA_KT * J_PER_KT_TNT

# Targets
TARGETS = {
    "Moon":   {"g": 1.62, "rho_t": 2.7},
    "Mercury":{"g": 3.70, "rho_t": 3.0},
    "Mars":   {"g": 3.71, "rho_t": 2.7},
    "Ceres":  {"g": 0.27, "rho_t": 2.2},
    "Venus":  {"g": 8.87, "rho_t": 2.9},
    "Earth":  {"g": 9.81, "rho_t": 2.7},
}

def crater_extended(
    rho_gcm3=3.0,
    D_km=10.0,
    v_kms=20.0,
    theta_deg=45.0,
    target="Earth",
    rho_t_gcm3=None,
    K_const=1.8,
    v_min_kms=5.0, v_max_kms=70.0,
    D_min_km=0.1, D_max_km=2000.0
):
    rho = rho_gcm3 * G_PER_CM3_TO_KG_PER_M3
    R = 0.5 * D_km * KM_TO_M
    V = (4/3)*math.pi*R**3
    m = rho * V
    v = v_kms * KMS_TO_MS
    E = 0.5 * m * v**2
    kt = E / J_PER_KT_TNT
    Mt = E / J_PER_MT_TNT
    hiro = E / J_PER_HIRO

    g = TARGETS[target]["g"]
    rho_t = rho_t_gcm3 if (rho_t_gcm3 and rho_t_gcm3>0) else TARGETS[target]["rho_t"]

    theta = math.radians(theta_deg)
    sin_term = max(math.sin(theta), 1e-6)
    D_crater_km = (K_const
                   * (rho_gcm3/max(rho_t,1e-6))**(1/3)
                   * (sin_term)**(1/3)
                   * (g**(-0.22))
                   * (v_kms**0.44)
                   * (D_km**0.78))

    print("=== Extended crater model (up to basin scale) ===")
    print(f"Target: {target} | g = {g:.2f} m/s² | ρ_t = {rho_t:.2f} g/cm³")
    print(f"Impactor: ρ = {rho_gcm3:.2f} g/cm³, D = {D_km:.1f} km, v = {v_kms:.1f} km/s, θ = {theta_deg:.1f}°")
    print(f"Energy: {E:.3e} J → {Mt:.3e} Mt TNT (~{hiro:.3e} Hiroshima)")
    print(f"Estimated crater diameter ≈ {D_crater_km:.1f} km (± factor ~2)")
    print("Caution: For craters > ~500 km, this scaling is approximate — real basins involve planetary-scale processes.")

    # Plot 1: Energy vs speed
    vgrid_kms = np.linspace(v_min_kms, v_max_kms, 300)
    E_v = 0.5 * m * (vgrid_kms*KMS_TO_MS)**2
    plt.figure(figsize=(6,4))
    plt.plot(vgrid_kms, E_v/J_PER_MT_TNT)
    plt.scatter([v_kms], [E/J_PER_MT_TNT], s=40)
    plt.xlabel("Impact speed (km/s)")
    plt.ylabel("Energy (Mt TNT)")
    plt.title("Energy vs speed (marker = chosen v)")
    plt.yscale("log")
    plt.grid(True, linestyle=":")
    plt.show()

    # Plot 2: Crater diameter vs impactor diameter
    Dgrid_km = np.linspace(D_min_km, D_max_km, 400)
    Dcr_grid = (K_const
                * (rho_gcm3/max(rho_t,1e-6))**(1/3)
                * (sin_term)**(1/3)
                * (g**(-0.22))
                * (v_kms**0.44)
                * (Dgrid_km**0.78))
    plt.figure(figsize=(6,4))
    plt.plot(Dgrid_km, Dcr_grid)
    plt.scatter([D_km], [D_crater_km], s=40)
    plt.xlabel("Impactor diameter (km)")
    plt.ylabel("Crater diameter (km)")
    plt.title("Crater size vs impactor size (marker = chosen D)")
    plt.yscale("log")
    plt.grid(True, linestyle=":")
    plt.show()

    return {"mass_kg": m, "energy_J": E, "energy_MtTNT": Mt, "crater_diam_km": D_crater_km}

# UI
interact(
    crater_extended,
    rho_gcm3=FloatSlider(value=3.0, min=0.5, max=8.0, step=0.1, description="ρ_i (g/cm³)"),
    D_km=FloatSlider(value=10.0, min=0.1, max=2000.0, step=1.0, description="D_imp (km)"),
    v_kms=FloatSlider(value=20.0, min=1.0, max=70.0, step=0.1, description="v (km/s)"),
    theta_deg=FloatSlider(value=45.0, min=5.0, max=90.0, step=1.0, description="θ (deg)"),
    target=Dropdown(options=list(TARGETS.keys()), value="Earth", description="Target"),
    rho_t_gcm3=FloatSlider(value=2.7, min=1.5, max=3.5, step=0.01, description="ρ_t override"),
    K_const=FloatSlider(value=1.8, min=0.5, max=3.0, step=0.05, description="K (scaling)"),
    v_min_kms=FloatSlider(value=5.0, min=0.5, max=50.0, step=0.5, description="Plot v_min"),
    v_max_kms=FloatSlider(value=70.0, min=5.0, max=100.0, step=1.0, description="Plot v_max"),
    D_min_km=FloatSlider(value=0.1, min=0.01, max=10.0, step=0.1, description="Plot D_min"),
    D_max_km=FloatSlider(value=2000.0, min=10.0, max=2000.0, step=10.0, description="Plot D_max"),
);

interactive(children=(FloatSlider(value=3.0, description='ρ_i (g/cm³)', max=8.0, min=0.5), FloatSlider(value=1…